In [1]:
import os
import sys
from functools import partial
from operator import itemgetter
from pathlib import Path

import requests
from dotenv import load_dotenv
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

load_dotenv()

ROOT_DIR = Path.cwd().parent.absolute()

In [2]:
def get_customer_manager_id(googleads_service, customer_service):
    # A collection of customer IDs to handle.
    seed_customer_ids = []
    customer_resource_names = (
        customer_service.list_accessible_customers().resource_names
    )
    for customer_resource_name in customer_resource_names:
        customer_id = googleads_service.parse_customer_path(customer_resource_name)[
            "customer_id"
        ]
        seed_customer_ids.append(customer_id)
    return seed_customer_ids


def get_customer_client_id(googleads_service, seed_customer_ids):
    # Creates a query that retrieves all child accounts of the manager
    # specified in search calls below.
    query = """
        SELECT
            customer_client.client_customer,
            customer_client.level,
            customer_client.manager,
            customer_client.descriptive_name,
            customer_client.currency_code,
            customer_client.time_zone,
            customer_client.id
        FROM customer_client
        WHERE customer_client.level <= 1"""

    for seed_customer_id in seed_customer_ids:
        # Performs a breadth-first search to build a Dictionary that maps
        # managers to their child accounts (customerIdsToChildAccounts).
        unprocessed_customer_ids = [seed_customer_id]
        customer_ids_to_child_accounts = dict()
        root_customer_client = None

        while unprocessed_customer_ids:
            customer_id = unprocessed_customer_ids.pop(0)
            response = googleads_service.search(customer_id=customer_id, query=query)

            # Iterates over all rows in all pages to get all customer
            # clients under the specified customer's hierarchy.
            for googleads_row in response:
                customer_client = googleads_row.customer_client

                # The customer client that with level 0 is the specified
                # customer.
                if customer_client.level == 0:
                    if root_customer_client is None:
                        root_customer_client = customer_client
                    continue

                # For all level-1 (direct child) accounts that are a
                # manager account, the above query will be run against them
                # to create a Dictionary of managers mapped to their child
                # accounts for printing the hierarchy afterwards.
                if customer_id not in customer_ids_to_child_accounts:
                    customer_ids_to_child_accounts[customer_id] = []

                customer_ids_to_child_accounts[customer_id].append(customer_client)

                if customer_client.manager:
                    # A customer can be managed by multiple managers, so to
                    # prevent visiting the same customer many times, we
                    # need to check if it's already in the Dictionary.
                    if (
                        customer_client.id not in customer_ids_to_child_accounts
                        and customer_client.level == 1
                    ):
                        unprocessed_customer_ids.append(customer_client.id)

    client_customer_ids = list(
        map(
            lambda x: x.id if not x.manager else None,
            customer_ids_to_child_accounts[seed_customer_id],
        )
    )

    return client_customer_ids

In [3]:
client = GoogleAdsClient.load_from_env()

# Gets instances of the GoogleAdsService and CustomerService clients.
googleads_service = client.get_service("GoogleAdsService")
customer_service = client.get_service("CustomerService")

# A collection of customer IDs to handle.
seed_customer_ids = get_customer_manager_id(googleads_service, customer_service)


customer_manager_id = os.getenv("GOOGLE_ADS_LOGIN_CUSTOMER_ID")

In [4]:
client_customer_ids = get_customer_client_id(googleads_service, seed_customer_ids)